# Imports

## Import Lib

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import re
import time

In [ ]:
start_time = time.time()

## Import Csv

In [ ]:
chunksize = 10 ** 3  # Chunksize de 1000 lignes
filename = 'products.csv'
nan_rates = []

try:
    # Lecture du fichier CSV par chunks
    reader = pd.read_csv(filename, chunksize=chunksize, sep='\t')

    for chunk in reader:
        # Calculer le taux de NaN pour chaque chunk
        nan_rates.append(chunk.isna().mean(axis=0))

except pd.errors.ParserError as e:
    print(f"ParserError: {e}")

# Calcul des taux moyens de remplissage des chunks
if nan_rates:
    inds_nan_rate = pd.concat(nan_rates, axis=1).mean(axis=1)

    # Sélection des colonnes à charger en fonction des taux de NaN
    cols_to_load = inds_nan_rate[inds_nan_rate < 0.7].index.tolist()

    # Chargement des données filtrées
    df = pd.read_csv(filename, sep='\t', usecols=cols_to_load)
    df = df.drop(['url','creator','created_datetime','food_groups_tags','food_groups_en','states_tags','sodium_100g','states_en','last_image_datetime','last_modified_t','last_modified_datetime','created_t','last_modified_by','main_category_en','image_url','image_small_url','last_updated_t','last_image_t','brands_tags','last_updated_datetime','categories_en','categories_tags','countries_tags','countries_en'], axis=1)

    # Affichage de la première partie du dataframe pour vérifier
    print(df.head())

# Information de base

In [ ]:
df.info

In [ ]:
df.head(5)

In [ ]:
df.dtypes

# Nettoyage Données

## Suppression cyrillique

In [ ]:
def detect_cyrillic(df, column):
    cyrillic_indices = []
    cyrillic_pattern = re.compile(r'[\u0400-\u04FF]+')  # Unicode Cyrillic characters

    for index, value in df[column].items():
        if isinstance(value, str) and cyrillic_pattern.search(value):
            cyrillic_indices.append(index)

    return cyrillic_indices

cyrillic_indices = detect_cyrillic(df, 'product_name')
df = df.drop(index=cyrillic_indices).reset_index(drop=True)
num_rows_removed = len(cyrillic_indices)
print(f"\nNombre de lignes supprimées : {num_rows_removed}")

print("\nDataFrame après suppression des lignes avec des caractères cyrilliques :")
print(df)

## Doublons

In [ ]:
for column in df.columns:
    doublons = df[column].duplicated().sum()
    print(f"Column: {column}")
    print(doublons)

### Suppression et verification des doublons dans 'code'

In [ ]:
df = df.drop_duplicates(subset='code')
for column in df.columns:
    doublons = df[column].duplicated().sum()
    print(f"Column: {column}")
    print(doublons)

In [ ]:
df.isna().any()

In [ ]:
df.isna().mean()

In [ ]:
valeurs_manq = df.isna().mean().sort_values(ascending=False)
sns.barplot(x=valeurs_manq.values, y=valeurs_manq.index);

## Detection, modification et suppression des valeurs supérieures a 100 pour 100gr

In [ ]:
def count_responses(results_df):
    return results_df.shape[0]

initial_size = df.shape[0]

columns_to_check = [
    'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g',
    'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g','fruits-vegetables-nuts-estimate-from-ingredients_100g'
]

results = []

for column in columns_to_check:
    # Convertir les valeurs de la colonne en float (si possible)
    df.loc[:, column] = pd.to_numeric(df[column], errors='coerce')
    values_above_100 = df[df[column] > 100][column]
    for idx, value in values_above_100.items():
        results.append((df.loc[idx, 'product_name'], column, value))

results_df = pd.DataFrame(results, columns=['product_name', 'Column', 'Value'])

print("Produits avec des valeurs supérieures à 100 :")
print(results_df)

# Compter le total de réponses
total_responses = count_responses(results_df)
print(f"Total de réponses trouvées : {total_responses}")

### Affichage valeur unique product_name

In [ ]:
unique_product_counts = df['product_name'].value_counts()
print("Valeurs uniques dans la colonne 'product_name' et leur nombre :")
print(unique_product_counts)

## Remplacement des >100 par la moyenne en fonction de product name

In [ ]:
columns_to_check = [
    'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g',
    'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g','fruits-vegetables-nuts-estimate-from-ingredients_100g'
]

# Convertir les valeurs des colonnes en float
df[columns_to_check] = df[columns_to_check].apply(pd.to_numeric, errors='coerce')

# Calculer les moyennes des colonnes en fonction de 'product_name'
means = df.groupby('product_name')[columns_to_check].transform('mean')

# Remplacer les valeurs supérieures à 100 par la moyenne correspondante
for column in columns_to_check:
    df[column] = np.where(df[column] > 100, means[column], df[column])

# Vérifier les valeurs supérieures à 100 après correction
values_above_100 = df[columns_to_check] > 100
count_above_100 = values_above_100.sum().sum()
print(f"Nombre de valeurs supérieures à 100 après correction : {count_above_100}")

# Supprimer les lignes contenant des valeurs supérieures à 100
df.drop(index=df[values_above_100.any(axis=1)].index, inplace=True)
print("DataFrame après suppression des valeurs supérieures à 100 :")
print(df)

## Iterative imputer dans les valeurs NaN 

In [ ]:
df[columns_to_check] = df[columns_to_check].apply(pd.to_numeric, errors='coerce')

modified_count = 0

# Utiliser IterativeImputer pour remplacer les NaN
imputer = IterativeImputer(random_state=0)
df_imputed = imputer.fit_transform(df[columns_to_check])

# Compter les valeurs modifiées
for col in columns_to_check:
    modified_count += df[col].isna().sum() - np.isnan(df_imputed[:, columns_to_check.index(col)]).sum()

# Remplacer les valeurs NaN dans le DataFrame d'origine
df[columns_to_check] = df_imputed

# Afficher le DataFrame final après imputation
print("DataFrame après imputation itérative :")
print(df)

# Afficher le nombre de valeurs modifiées
print(f"Nombre de valeurs modifiées : {modified_count}")

In [ ]:
for column in df.columns:
    if column == 'fruits-vegetables-nuts-estimate-from-ingredients_100g': 
        unique_values = df[column].unique()
        print(f"Colonne '{column}':")
        for value in unique_values:
            print(f"  - {value}")
        print()

## Suppression des NaN dans product_name

In [ ]:
nombre_lignes_avant = df.shape[0]
df.dropna(subset=['product_name'], inplace=True)
nombre_lignes_apres = df.shape[0]
nombre_lignes_supprimees = nombre_lignes_avant - nombre_lignes_apres
print(f"\nNombre de lignes supprimées: {nombre_lignes_supprimees}")

### Pourcentage des 0 et NaN restant

In [ ]:
zero_percentages = (df == 0).sum() / df.shape[0] * 100
nan_percentages = df.isna().sum() / df.shape[0] * 100
print("Pourcentage des valeurs égales à 0 :")
print(zero_percentages)
print("\nPourcentage des NaN :")
print(nan_percentages)

In [ ]:
df.head(5)

## Modifications valeurs negatives

In [ ]:
columns_to_check = [
    'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g',
    'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g'
]

# Vérifier, compter et afficher les valeurs négatives
negative_values = {}

for column in columns_to_check:
    negative_rows = df[df[column] < 0]
    count_negatives = negative_rows.shape[0]
    if count_negatives > 0:
        negative_values[column] = count_negatives
        print(f"Colonne '{column}':")
        print(negative_rows[['product_name', column]])
        print(f"Nombre de valeurs négatives dans '{column}': {count_negatives}")
        print()

# Convertir les valeurs négatives en valeurs positives dans le DataFrame principal
df[columns_to_check] = df[columns_to_check].abs()

# Afficher le résumé des valeurs négatives
print("\nRésumé des valeurs négatives converties :")
for column, count in negative_values.items():
    print(f"Colonne '{column}': {count} valeurs négatives converties en positives")

In [ ]:
df.head(5)

# Timer

In [ ]:
end_time = time.time()
execution_time = end_time - start_time

print(f"Temps d'exécution total du notebook : {execution_time:.4f} secondes")